In [1]:
from pathlib import Path
from bs4 import BeautifulSoup
import requests
import csv 
import re
from langdetect import detect

In [14]:
f = open("homework.txt", "w")

for i in range(1,301):
    page = requests.get("https://www.goodreads.com/list/show/1.Best_Books_Ever?page=" + str(i))
    soup = BeautifulSoup(page.content, features="lxml")
    links = soup.find_all('a', itemprop='url', class_='bookTitle')
    for link in links:
        fullLink = link.get('href')
        f.write('https://www.goodreads.com' + fullLink + '\n')
f.close()

In [ ]:
path = './storage/'


for i in range(1, 29701):
    
    folderName = "folder-" + str(i) + "/"
    fileName = "article_" + str(i) + ".html"
    
    f = open("homework3.txt", "r")
    url = f.readlines()[i-1]
    
    Path(path + folderName).mkdir(parents=True, exist_ok=True)

    page = requests.get(url)
    code = str(page.text)

    with open(path + folderName + fileName, "w", encoding="utf-8") as z:
        z.write(code)

    z.close()
    f.close()

In [5]:
data = ['bookTitle', 'bookSeries', 'bookAuthors', 'ratingValue', "ratingCount",\
        "reviewCount", "plot", "numberOfPages", "publishingDate", "characters", "setting", "url"]

personalPath = "C:/Users/giogi/Data Science/Algorithmic Methods of Data Mining/storage/folder-"

for i in range(9901, 9934): #9901
    with open(personalPath + str(i) + "/article_" + str(i) + ".html", 'rb') as html: 
        soup = BeautifulSoup(html,"html.parser")
    #plot = soup.find('div',id='description').text.strip()  
    try:
        plot = soup.find('div',id='description').text.strip()
        if detect(plot)=='en':
            lista=[]

            #title
            try:
                lista.append(soup.find('h1').text.strip())
            except:
                lista.append('')

                #bookseries
            try:
                lista.append(soup.find('h2',id='bookSeries').text.strip())
            except:
                lista.append('')

                #author name
            try:
                lista.append(soup.find('a',class_='authorName').text.strip())
            except:
                lista.append('')

                #rating value
            try:
                lista.append(soup.find('span', itemprop='ratingValue').text.strip())
            except:
                lista.append('')

                #ratingCount
            try:
                lista.append(soup.find_all('a',class_='gr-hyperlink',href='#other_reviews')[0]\
                             .text.strip().replace('\r', '').replace('\n', '').split()[0])
            except:
                lista.append('')

                #reviewCount
            try:
                lista.append(soup.find_all('a',class_='gr-hyperlink',href='#other_reviews')[1]\
                             .text.strip().replace('\r', '').replace('\n', '').split()[0])
            except:
                lista.append('')

                #plot
            try:
                if plot[-7:] == '...more':
                    lista.append(soup.find('div',id='description').contents[3].text)
                else:
                    lista.append(plot)
            except:
                lista.append('')

                #number of pages
            try:
                lista.append(soup.find('span', itemprop='numberOfPages').text.strip().split()[0])
            except:
                lista.append('')

                #Publishing Date
            try:
                a=soup.find_all('div', class_='row')[1].text
                match_obj = re.split('Published', re.split('by', a)[0])[1]
                lista.append(match_obj.strip())
            except:
                lista.append('')

                #characters
            try:
                l1=[]
                for d in soup.find_all('a',href=re.compile(r'/characters/*')):
                    l1.append(d.text)
                    s1=",".join(l1)
                lista.append(s1)
            except:
                lista.append('')

                #setting
            try:
                l2=[]
                for e in soup.find_all('a',href=re.compile(r'/places/*')):
                    l2.append(e.text)
                    s2=",".join(l2)
                lista.append(s2)
            except:
                lista.append('')

                #URL
            lista.append(soup.find('link')['href'].strip())

            path = personalPath + str(i) + '/article_' + str(i)+ '.tsv'

            with open(path, 'w', newline='',encoding="utf-8") as f_output:
                tsv_output = csv.writer(f_output, delimiter='\t')
                tsv_output.writerow(data)
                tsv_output.writerow(lista)
                f_output.close()
        
        else:
            print('This book is not in english: '+ str(i))
            
    except:
        print('Missing plot for book: '+ str(i))

This book is not in english: 9932


In [29]:
import nltk
import string
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer 
from nltk.tokenize import RegexpTokenizer

from scipy import spatial

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\giogi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
def generateTermId(word):
    term_id = ''
    for c in word:
        term_id += str(string.ascii_lowercase.index(c.lower()))
    
    return int(term_id)

def getIds(keywords):
    term_ids = []
    for word in keywords:
        try:
            term_ids.append(voc[word])
        except:
            pass

    return term_ids

def getBooks(term_ids):
    books = []
    for ids in term_ids:
        books.append(set(diz[ids]))

    return set.intersection(*books)

In [40]:
# As suggested by the homework, we should create a file with all the following data. 
# In my opinion, better to create three files: one for the vocabulary (voc), and two for each database (diz) to be created
# for searchEngin1 and searchEngine2

books=open("C:/Users/giogi/Data Science/Algorithmic Methods of Data Mining/books.tsv", 'r')    # TO BE MODIFIED
l=books.readlines()
i=1
ps = PorterStemmer() 
l1=[]

diz={}
voc = {}

for line in l:
    line1=line.split('\t')
    tokenizer = RegexpTokenizer(r"[a-zA-Z]+")
    text_tokens = tokenizer.tokenize(line1[6])

    for word in set(text_tokens):
        if not word in stopwords.words():
            a = ps.stem(word.lower())
            term_id = generateTermId(a)
            
            if a not in voc:
                voc[a] = term_id
            if term_id not in diz:
                diz[term_id] = [i]
            else:
                if i not in diz[term_id]:
                    diz[term_id].append(i)

    i+=1

    if i==100:            # TO BE REMOVED
        break            # TO BE REMOVED

In [41]:
def searchEngine1(voc, diz):
    
    query = input()
    keywords = query.split()
    
    term_ids = getIds(keywords)
    books = getBooks(term_ids)
    
    return books

In [42]:
searchEngine1(voc, diz)

love win


{1, 3, 6, 10}

In [44]:
voc

{'live': 118214,
 'nation': 1301981413,
 'contend': 21413194133,
 'capitol': 20158191411,
 'place': 1511024,
 'eighteen': 4867194413,
 'love': 1114214,
 'send': 184133,
 'forc': 514172,
 'surround': 182017171420133,
 'outli': 142019118,
 'mean': 124013,
 'hunger': 720136417,
 'choic': 271482,
 'katniss': 100191381818,
 'fight': 586719,
 'panem': 15013412,
 'without': 228197142019,
 'north': 131417197,
 'the': 1974,
 'surviv': 18201721821,
 'everdeen': 42141734413,
 'second': 184214133,
 'district': 381819178219,
 'life': 11854,
 'sister': 1881819417,
 'forward': 51417220173,
 'boy': 11424,
 'year': 244017,
 'in': 813,
 'regard': 17460173,
 'dead': 3403,
 'cruel': 21720411,
 'mother': 1214197417,
 'line': 118134,
 'lie': 1184,
 'keep': 104415,
 'sure': 1820174,
 'old': 14113,
 'win': 22813,
 'make': 120104,
 'particip': 150171982815,
 'natur': 130192017,
 'annual': 131320011,
 'weight': 22486719,
 'sixteen': 18823194413,
 'younger': 241420136417,
 'shine': 1878134,
 'harsh': 7017187,
 '